In [8]:
# Nhập các thư viện cần thiết trong quá trình thực hiện
import vnstock as vnst
import pandas as pd
import numpy as np
import cvxpy as cp

In [9]:
def get_combined_df(list_stock):    
    symbols = list_stock
    start_date = "2023-06-14"
    end_date = "2024-02-01"

    historical_data = {}

    for symbol in symbols:
        historical_data[symbol] = vnst.stock_historical_data(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date,
            type=type
        )

    # Gộp các DataFrame trong danh sách theo cột 'close'
    df_combined = pd.concat([historical_data[symbol]['close'] for symbol in symbols], axis=1)

    # Đặt tên cột mới
    df_combined.columns = symbols
    return df_combined

In [10]:
def covariance_matrix(corr_matrix, volatility):
    n = len(volatility)
    cov_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            cov_matrix[i, j] = corr_matrix.iloc[i, j] * volatility[i] * volatility[j]  # Changed corr_matrix[i, j] to corr_matrix.iloc[i, j]
    return cov_matrix

def objective(x, cov_matrix):
    return cp.quad_form(x, cov_matrix)

def cal_weight(df_combined, list_stock):
    days_return_3days = df_combined / df_combined.shift(3) - 1
    days_return_3days.dropna(inplace=True)
    returns_20 = days_return_3days.head(20).mean()
    returns_60 = days_return_3days.tail(60).mean()
    volatility = np.std(days_return_3days.tail(20))
    corr_matrix = df_combined.corr()

    cov_matrix = covariance_matrix(corr_matrix, volatility)
    num_assets = cov_matrix.shape[0]
    x = cp.Variable(num_assets)
    objective_func = cp.Minimize(cp.quad_form(x, cov_matrix))
    constraints = [
        cp.sum(x) == 1,  # a + b + c + d + e = 1
        x >= 0.05,  # a, b, c, d, e >= 0.05
        cp.matmul(returns_20.values, x) >= 0,  # Lớn hơn hoặc bằng 0
        cp.matmul(returns_60.values, x) >= 0  # Lớn hơn hoặc bằng 0
    ]
    problem = cp.Problem(objective_func, constraints)
    weight_df = problem.solve()
    weight_df = pd.DataFrame({'weight': x.value}, index=list_stock).round(4)
    return weight_df

In [11]:
list_stock = ['SSI','HPG','VND','GAS','FPT']
df_combined =  get_combined_df(list_stock)
weight_df = cal_weight(df_combined, list_stock)
weight_df.head()

Error in API response {"status":500,"code":"INTERNAL_SERVER_ERROR","message":"code=404, message=Not Found"}
 



UnboundLocalError: local variable 'df' referenced before assignment